<a href="https://colab.research.google.com/github/hanubv/ML_Zoo/blob/master/Logistic_regeression_by_torch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import numpy as np
from torch.utils.data import DataLoader
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [ ]:
#Creating customDataset() class to get the data
class CustomDataset():

    def __init__(self, data, targets):
        self.data = data
        self.targets  = targets

    def __len__(self):
        return self.data.shape[0]

    def __getitem__(self, idx):
        current_data = self.data[idx]
        current_target = self.targets[idx]

        return {
            "x" : torch.tensor(current_data, dtype = torch.float),
            "y" : torch.tensor(current_target, dtype = torch.float)
        }


In [ ]:
#Creating Dataset and accessing the sampele and labels into the variable data and targets
data, target = make_classification(n_samples = 1000)
train_data, test_data, train_targets, test_targets = train_test_split(data,target, stratify = target) #Data splitting for training and testing purpose

In [ ]:
print(train_data)
print(train_targets)
print(test_data)
print(test_targets)


[[ 0.27084224 -0.08396255 -0.53172611 ...  0.570051    0.79457196
   0.80237367]
 [-0.59184569 -0.7387309  -0.22554775 ... -1.65915857  0.41121541
   1.53891148]
 [ 0.11090164  0.14086789  1.39230214 ...  0.19029896  1.41007501
  -1.21272073]
 ...
 [-0.16562591  0.42623414 -0.90384464 ... -0.00605438  0.89834866
  -1.44361025]
 [-0.64555231 -1.01887744  0.09683528 ... -0.32398259  0.86897586
  -1.34400609]
 [ 0.61449665 -0.64512819  2.09434442 ... -0.2601655  -2.30048663
  -1.05208064]]
[1 0 1 0 1 1 0 1 0 0 1 1 1 1 1 1 0 1 1 1 0 1 0 1 0 1 1 0 0 0 1 0 1 0 1 1 0
 0 1 1 0 0 1 0 1 0 1 0 0 1 1 0 1 0 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 1 0 1 1
 1 1 1 0 0 0 0 0 0 1 1 0 0 0 1 0 1 1 1 0 0 1 0 0 0 0 1 0 0 1 1 0 1 1 0 1 1
 1 0 0 1 1 0 0 0 0 0 0 1 0 1 0 0 1 0 0 0 1 0 1 0 0 0 1 1 0 0 1 0 0 1 0 0 0
 1 1 1 0 1 1 0 0 0 1 1 0 0 0 0 0 0 0 1 1 0 0 0 1 1 0 1 1 1 1 0 0 0 0 0 1 1
 1 0 0 1 0 0 1 0 0 0 1 0 0 0 1 1 0 0 1 1 1 1 0 1 1 1 1 1 1 1 0 1 0 1 0 1 0
 0 1 0 1 1 1 0 1 0 1 1 0 0 0 0 1 0 1 0 0 1 1 0 1 1 0 0 1 1

In [ ]:
#Loading data into our CustomDataset()
train_dataset = CustomDataset(train_data, train_targets )
test_dataset  = CustomDataset(test_data, test_targets)

In [ ]:
#Feeding the input data to DataLoader() to iterate through the dataset
train_dataloader = DataLoader(train_dataset, batch_size = 4, num_workers = 2)
test_dataloader = DataLoader(test_dataset, batch_size = 4, num_workers = 2)

In [ ]:
#Creating our linear model(y= mx+c)
model  = lambda x, w, b: torch.matmul(x, w)+b

In [ ]:
#Initializing wights, bias and learning rate
w = torch.randn(20,1, requires_grad = True)
b = torch.randn(1, requires_grad = True)
learning_rate = 0.001

In [ ]:
for epoch in range(10):
    epoch_loss = 0
    counter = 0
    for data in train_dataloader:
        x_train = data["x"]
        y_train = data["y"]
        output  = model(x_train, w, b)
        loss  = torch.mean((y_train.view(-1) - output.view(-1))**2)   #MSE loss function
        epoch_loss = epoch_loss + loss.item()
        #loss.requires_grad = True
        loss.backward()

        with torch.no_grad():
            w  = w - (learning_rate* w.grad)    #updating weights
            b  = b - (learning_rate * b.grad)   #updating bias

        w.requires_grad_(True)
        b.requires_grad_(True)
        counter = counter+1

    print(epoch, epoch_loss/counter)


0 0.10605667213453575
1 0.10605157279667068
2 0.10604892097453171
3 0.10604754662478065
4 0.10604686076019673
5 0.10604652833252987
6 0.10604636836301615
7 0.10604630575138838
8 0.10604628476690739
9 0.106046278069311


In [ ]:
#Performing operation on Test dataset
outputs = []
labels = []
with torch.no_grad():
    for data in test_dataloader:
        x_test = data["x"]
        y_test = data["y"]
        output = model(x_test, w, b)
        labels.append(y_test)
        outputs.append(output)

#Find Accuracy
auc = metrics.roc_auc_score(torch.cat(labels).view(-1), torch.cat(outputs).view(-1))
print(auc)
print(torch.cat(labels).view(-1), torch.cat(outputs).view(-1))


0.9400921658986174
tensor([1., 1., 0., 1., 1., 1., 0., 0., 0., 1., 0., 1., 0., 1., 1., 0., 0., 1.,
        1., 0., 1., 1., 1., 0., 1., 1., 0., 0., 0., 0., 0., 1., 1., 1., 0., 1.,
        1., 0., 0., 1., 0., 1., 0., 1., 1., 1., 0., 1., 0., 0., 0., 1., 0., 1.,
        1., 1., 0., 1., 0., 1., 0., 1., 1., 0., 0., 1., 0., 0., 1., 0., 0., 0.,
        1., 1., 1., 1., 0., 0., 1., 0., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0.,
        0., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0.,
        1., 1., 0., 1., 0., 0., 1., 1., 0., 0., 0., 1., 0., 1., 0., 0., 1., 1.,
        0., 1., 1., 1., 0., 1., 1., 1., 1., 0., 1., 1., 0., 1., 1., 0., 0., 1.,
        1., 0., 1., 1., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 1., 0., 1.,
        1., 1., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 1., 1.,
        1., 1., 0., 1., 0., 0., 0., 1., 1., 1., 0., 0., 1., 1., 0., 0., 0., 1.,
        1., 1., 0., 1., 1., 0., 0., 0., 1., 0., 0., 0., 0., 1., 1., 0., 1., 1.,
        0., 0., 0., 1